In [2]:
import pandas as pd
import csv

In [2]:
import pandas as pd
import datetime

# Load your dataset 
df = pd.read_csv('~/Downloads/Farsi-emails.csv')

# Convert datetime column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Define the date range
start_date = datetime.datetime(2021, 9, 27, 0, 0, 0)
end_date = datetime.datetime(2021, 9, 30, 23, 59, 59)

# Filter the dataframe based on the date range
filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)].copy()

# Remove rows containing specific text
filtered_df.loc[:, 'full_text'] = filtered_df['full_text'].astype(str)
filtered_df = filtered_df[~filtered_df['full_text'].str.contains('Your Zimbra Mailbox Has Exceeded It Quota/')]

# Function to format the row as required
def format_row(row):
    try:
        frm = ', '.join([f"name: {d['name']}, email: {d['email']}" for d in eval(row['frm'])])
        too = ', '.join([f"name: {d['name']}, email: {d['email']}" for d in eval(row['too'])])
    except:
        frm = 'Invalid format'
        too = 'Invalid format'
    formatted_text = (
        f"ID: {row['ID']}\n"
        f"Date: {row['datetime']}\n"
        f"From: {frm}\n"
        f"To: {too}\n\n"
        f"Subject: {row['subject']}\n\n"
        f"Full Text:\n{row['full_text']}"
    )
    return formatted_text

# Apply the formatting function
filtered_df['merged_column'] = filtered_df.apply(format_row, axis=1)

# Save to a single-column CSV
single_column_filtered_df = filtered_df[['merged_column']]
single_column_filtered_df.to_csv('new_single_column_filtered.csv', index=False)

# Output the number of emails during the period
num_emails = filtered_df.shape[0]
print("Number of emails during the period:", num_emails)


Number of emails during the period: 132


In [3]:
import pandas as pd
import os
from docx import Document

# Load the single column CSV file
df = single_column_filtered_df

# Define the output directory
output_dir = os.path.expanduser('~/Downloads/farsi-emails')
os.makedirs(output_dir, exist_ok=True)

# Function to create a DOCX file for each row
def create_docx(row):
    # Create a new Document
    doc = Document()
    
    # Add the content to the Document
    doc.add_paragraph(row['merged_column'])
    
    # Extract the ID from the merged column
    email_id = row['merged_column'].split('\n')[0].split(': ')[1]
    
    # Save the Document with the ID as the filename
    filename = "ID_" + email_id + ".docx"
    doc.save(os.path.join(output_dir, filename))

# Apply the function to each row
df.apply(create_docx, axis=1)

print(f"DOCX files created in {output_dir}")


DOCX files created in /home/pansear/Downloads/farsi-emails
